# **Data Engineer Project - ETL PIPELINE**
***This project provided by IBM - Coursera***

# Objectives
In this project we will:

+ Run the ETL process
+ Extract bank and market cap data from the JSON file bank_market_cap.json
+ Transform the market cap currency using the exchange rate data
+ Load the transformed data into a seperate CSV

# Imports
Let's download the files that we need and import python libraries wanted for this project.

> **Downloading the files**

In [1]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_1.json
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_2.json
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Final%20Assignment/exchange_rates.csv

print('Setup Complete')

--2022-08-16 00:33:30--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_1.json
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 198.23.119.245
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|198.23.119.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2815 (2.7K) [application/json]
Saving to: ‘bank_market_cap_1.json’

bank_market_cap_1.j 100%[===================>]   2.75K  --.-KB/s    in 0s      

2022-08-16 00:33:31 (213 MB/s) - ‘bank_market_cap_1.json’ saved [2815/2815]

--2022-08-16 00:33:32--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Tran

> **Importing python libraries**

In [2]:
import glob
import pandas as pd
from datetime import datetime

# EXTRACT
> **Extracting from JSON file.**

In [3]:
def extract_from_json(file):
    # Reading the JSON file into a dataframe
    df = pd.read_json(file)
    
    return df

> **Extract Function.**
+ Define the extract function that finds JSON files and calls the function created above to extract data from them. Store the data in a pandas dataframe.

In [4]:
def extract():
    # The extracted data
    df = pd.DataFrame()
    
    # Process JSON files
    for file in glob.glob('*.json'):
        df = df.append(extract_from_json(file), ignore_index=True)
    
    return df     

> Loading the file **exchange_rates.csv** as a dataframe and finding the **exchange rate** for British pounds with the symbol **GBP**, storing it in the variable ***exchange_rate***, we will need it later.

In [5]:
# Loading the data
df = pd.read_csv('exchange_rates.csv', index_col=0)
df.head()

,Rates
AUD,1.297088
BGN,1.608653
BRL,5.409196
CAD,1.271426
CHF,0.886083


In [6]:
# Finding the exchange rate for British pounds with the symbol GBP
exchange_rate = df.loc['GBP'][0]
print(f'The exchange rate is {exchange_rate:.6f}')

The exchange rate is 0.732398


# TRANSFORM
> **Writing a transform function that:**

+ Changes the '**Market Cap (US Billion)**' column from **USD** to **GBP**
+ Rounds the '**Market Cap (US Billion)**' column to **3 decimal places**
+ Rename '**Market Cap (US Billion)**' to '**Market Cap (GBP Billion)**'

In [7]:
def transform(data):
    # Changing from USD to GBP
    data['Market Cap (US$ Billion)'] = round(exchange_rate * data['Market Cap (US$ Billion)'], 3)
    
    # Renaming the column
    data.rename(columns = {'Market Cap (US$ Billion)': 'Market Cap (GBP$ Billion)'}, inplace=True)
    
    return data

# LOAD
> **Create a function that takes a dataframe and load it to a CSV.**

In [8]:
def load(target_file, data):
    # Loading data to CSV file which is target_file.
    data.to_csv(target_file, index=False)

# LOGGING FUNCTION

In [9]:
def log(msg):
    # Year-Month-Day-Hour-Minute-Second
    time_format = '%Y-%h-%d-%H:%M:%S'
    
    # get current time
    now = datetime.now()
    time = now.strftime(time_format)
    
    with open('./logfile.txt','a') as f:
        f.write('- ' + time + ' , ' + msg + '.\n')
    

# Running the ETL Process
> **Logging the process accordingly:**

In [10]:
log("ETL Job Started")

> **Extract**
+ Use the function extract, and print the first 5 rows

In [11]:
log("Extract phase Started")
# Extracting data
extracted_data = extract()

# Printing the first five rows
extracted_data.head()

,Name,Market Cap (US$ Billion)
0,JPMorgan Chase,390.934
1,Industrial and Commercial Bank of China,345.214
2,Bank of America,325.331
3,Wells Fargo,308.013
4,China Construction Bank,257.399


In [12]:
log("Extract phase Ended")

> **Transform**
+ Use the function transform, and print the first 5 rows

In [13]:
log("Transform phase Started")
# Transforming the data
transformed_data = transform(extracted_data)

# Printing the first 5 rows
transformed_data.head()

,Name,Market Cap (GBP$ Billion)
0,JPMorgan Chase,286.319
1,Industrial and Commercial Bank of China,252.834
2,Bank of America,238.272
3,Wells Fargo,225.588
4,China Construction Bank,188.519


In [14]:
log("Transform phase Ended")

> **Load**
+ Loading data into a **CSV file**

In [15]:
# The file
target = './loaded_data.csv'

# Loading the data
log("Load phase Started")
load(target, extracted_data)
log("Load phase Ended")

# **Checking**
> Checking if the data saved correctly in the file ***loaded_data.csv***

In [16]:
saved_data = pd.read_csv('./loaded_data.csv')
saved_data.head()

,Name,Market Cap (GBP$ Billion)
0,JPMorgan Chase,286.319
1,Industrial and Commercial Bank of China,252.834
2,Bank of America,238.272
3,Wells Fargo,225.588
4,China Construction Bank,188.519


> Reading the **log file**

In [17]:
log_file = open('./logfile.txt', 'r')
print(log_file.read())


- 2022-Aug-16-00:33:35 , ETL Job Started.
- 2022-Aug-16-00:33:35 , Extract phase Started.
- 2022-Aug-16-00:33:35 , Extract phase Ended.
- 2022-Aug-16-00:33:35 , Transform phase Started.
- 2022-Aug-16-00:33:35 , Transform phase Ended.
- 2022-Aug-16-00:33:35 , Load phase Started.
- 2022-Aug-16-00:33:35 , Load phase Ended.

